In [11]:
import random

class QuantumCircuitSampling:
    
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
        self.gate_options_without_cnot = ['pauli_x', 'pauli_y','pauli_z','rx_gate','ry_gate','rz_gate','phase','t','hadamard']
        self.rotation_gate_options = ['rx_gate', 'ry_gate', 'rz_gate']
        self.non_parametrized_gates = ['pauli_x', 'pauli_y','pauli_z', 'phase','t','hadamard']
    
    def generate_layer_without_entanglement(self):
       return [random.choice(self.gate_options_without_cnot) for wire in range(self.n_qubits)]
    
    def generate_rotation_layer(self):
        return [random.choice(self.rotation_gate_options) for wire in range(self.n_qubits)]
    
    def place_a_single_gate(self):
        return random.choice(self.gate_options_without_cnot) 
    
    def generate_disjoint_cnots(self):
        cnot_count_layer_one = 0
        cnot_count_layer_two = 0
        layer_one = []
        layer_two = []
        
        layer_two.append(self.place_a_single_gate())
        
        for wire in range(self.n_qubits - 1):
            if wire%2 == 0:
                layer_one.append(f'ctrl_{cnot_count_layer_one}')
                layer_one.append(f'trgt_{cnot_count_layer_one}')
                cnot_count_layer_one += 1
               
            else:
                layer_two.append(f'ctrl_{cnot_count_layer_two}')
                layer_two.append(f'trgt_{cnot_count_layer_two}')
                cnot_count_layer_two += 1
            
        if self.n_qubits%2 == 0:
            layer_two.append(self.place_a_single_gate())
        else:
            layer_one.append(self.place_a_single_gate())
        
        return layer_one, layer_two
    
    def generate_non_parametrized_layer(self):
        return [random.choice(self.non_parametrized_gates) for wire in range(self.n_qubits)]

    def generate_partially_entangled_layer(self):
        pairs = [(wire,wire+1) for wire in range(self.n_qubits-1)]
        #print(pairs)
        selected_pair = random.choice(pairs)
        control, target = selected_pair
        layer = ['ctrl_0' if wire == control else
                'trgt_0' if wire == target else
                self.place_a_single_gate()
                for wire in range(self.n_qubits)]
        
        return layer
    
    def generate_individual(self):
        ansatz = []
        n_layers = random.randint(4, 10)
        print(f'Generating {n_layers} layers...')
        depth = 0
        while depth <= n_layers:
            print(f'Generating layer #{depth}')
            gate_layer = []
            layer_type = random.randint(0,4)
            if layer_type == 0:
                layer = self.generate_layer_without_entanglement()
                ansatz.append(layer)
            elif layer_type == 1:
                layer_one, layer_two = self.generate_disjoint_cnots()
                ansatz.append(layer_one)
                ansatz.append(layer_two) 
                depth+=1
            elif layer_type == 2:
                layer = self.generate_rotation_layer()
                ansatz.append(layer)
            elif layer_type == 3:
                layer = self.generate_non_parametrized_layer()
                ansatz.append(layer)
            elif layer_type == 4:
                layer = self.generate_partially_entangled_layer()
                ansatz.append(layer)
                
            depth+=1
            
        
        return ansatz

    def _do(self, n_samples):
        print('where are you?')
        population = []
        
        for individual in range(n_samples):
            print(f'Generating individual...')
            population.append(self.generate_individual())
                           
        return population
    


In [12]:
sampling = QuantumCircuitSampling(4)
population = sampling._do(10)

where are you?
Generating individual...
Generating 8 layers...
Generating layer #0
Generating layer #1
Generating layer #2
Generating layer #3
Generating layer #4
Generating layer #6
Generating layer #7
Generating layer #8
Generating individual...
Generating 9 layers...
Generating layer #0
Generating layer #2
Generating layer #3
Generating layer #5
Generating layer #6
Generating layer #8
Generating layer #9
Generating individual...
Generating 6 layers...
Generating layer #0
Generating layer #1
Generating layer #2
Generating layer #4
Generating layer #6
Generating individual...
Generating 6 layers...
Generating layer #0
Generating layer #1
Generating layer #2
Generating layer #3
Generating layer #4
Generating layer #5
Generating layer #6
Generating individual...
Generating 9 layers...
Generating layer #0
Generating layer #1
Generating layer #2
Generating layer #4
Generating layer #6
Generating layer #7
Generating layer #8
Generating layer #9
Generating individual...
Generating 7 layers.

In [15]:
from itertools import combinations

ansatz_pairs = combinations(population, r=2)
len(ansatz_pairs)

TypeError: object of type 'itertools.combinations' has no len()

In [16]:
import torch
from math import sqrt 
import itertools
from ansatz_simulation_class import AnsatzSimulation
from collections import Counter
import math
from torch import inner

class RemoveEquivalentAnsätze:

    def __init__(self, n_tests, threshold_value, n_qubits):
        self.n_tests = n_tests
        self.threshold_value = threshold_value
        self.n_qubits = n_qubits
        self.circuit_ansatz = AnsatzSimulation(self.n_qubits)
        
        

    def generate_random_parameters(self, ansatz):
        gates = list(itertools.chain.from_iterable(ansatz.tolist()))
        gate_count = Counter(gates)
        n_params = gate_count['rx_gate'] + gate_count['ry_gate'] + gate_count['rz_gate']
        parameters = torch.rand(n_params)*math.pi

        return parameters

    def test_ansatz(self, ansatz, parameters, state_vector):
        return self.circuit_ansatz.simulate_circuit(input=state_vector, embedding_type='rx', ansatz_chromosome=ansatz, parameters=parameters, measure=False)
            
    # Check whether i calculate the inner product with just the real parts or should i include the imaginary ones as well
    def inner_product(self, state_vector_psi, state_vector_phi):
        return inner(state_vector_psi, state_vector_phi)

    def fidelity(self, state_vector_psi, state_vector_phi):
        return 1 - torch.einsum('i,i->i',state_vector_psi, state_vector_phi)**2


    def euclidian_distance(self, state_vector_psi, state_vector_phi):
        return sqrt(2 - 2*self.inner_product(state_vector_psi.real, state_vector_phi.real))

    # Decide which measure should be used in order to evaluate the

    def is_equal(self, ansatz_a, ansatz_b):
        print(type(ansatz_a))
        ansatz_a = ansatz_a.tolist()
        ansatz_b = ansatz_b.tolist()
        input_test = torch.rand(self.n_tests, self.n_qubits)
        params_a = self.generate_random_parameters(ansatz_a)
        params_b = self.generate_random_parameters(ansatz_b)
        
        output_a = [self.test_ansatz(ansatz_a, params_a, state_vector) for state_vector in input_test]
        output_b = [self.test_ansatz(ansatz_b, params_b, state_vector) for state_vector in input_test]
        output_pair = list(zip(output_a, output_b))
        
        fidelity_score = torch([self.fidelity(state_psi, state_phi) for (state_psi, state_phi) in output_pair])
        
        score = torch.mean(fidelity_score)
        
        return self.threshold_value > score[0]